In [1]:
import math
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
import pandas as pd
import seaborn as sns
from doe_xstock.database import SQLiteDatabase
from doe_xstock.utilities import split_lines

In [2]:
database_filepath = '/Users/kingsleyenweye/Desktop/INTELLIGENT_ENVIRONMENT_LAB/doe_xstock/database.db'
figures_directory = 'figures/'
schedule_data_directory = '../schedule_data'
database = SQLiteDatabase(database_filepath)

# Understanding the Metadata
***

In [51]:
# # numeric metadata
metadata = database.get_table('metadata')
columns_to_exclude = ['id', 'bldg_id', 'dataset_id','upgrade','metadata_index','in_county','in_puma','in_ashrae_iecc_climate_zone_2004', 'in_building_america_climate_zone', 'in_iso_rto_region', 'applicability', 'in_ahs_region', 'in_applicable','in_cec_climate_zone','in_census_division','in_census_division_recs','in_census_region','in_geometry_building_type_acs','in_geometry_building_type_height','in_geometry_building_type_recs','in_state','in_weather_file_longitude','in_weather_file_latitude','in_weather_file_city','in_nhgis_county_gisjoin','in_state_name','in_american_housing_survey_region','in_weather_file_2018','in_weather_file_tmy3','in_resstock_county_id','in_vacancy_status']
columns = [c for c in metadata.columns if c not in columns_to_exclude and pd.api.types.is_numeric_dtype(metadata[c])]
column_count = 6
row_count = math.ceil(len(columns)/column_count)
fig, axs = plt.subplots(row_count, column_count, figsize=(4*column_count,3*row_count))

for ax, column in zip(fig.axes, columns):
    ax.hist(metadata[column])
    ax.set_title(split_lines(column, line_character_limit=28,delimiter='_'))

plt.tight_layout()
plt.savefig(os.path.join(figures_directory,'travis_county_numeric_metadata_histogram.pdf'),transparent=True, bbox_inches='tight')
plt.close()

# non numeric metadata
columns = [c for c in metadata.columns if c not in columns_to_exclude and not pd.api.types.is_numeric_dtype(metadata[c])]
column_count = 4
row_count = math.ceil(len(columns)/column_count)
fig, axs = plt.subplots(row_count, column_count, figsize=(7*column_count,6*row_count))

for ax, column in zip(fig.axes, columns):
    plot_data = metadata.groupby(column).size().reset_index(name='count')
    x, y = list(range(plot_data.shape[0])), plot_data['count']
    ax.barh(x,y)
    ax.set_yticks(x)
    ax.set_yticklabels(plot_data[column].to_list())
    ax.set_title(split_lines(column, line_character_limit=28,delimiter='_'))

plt.tight_layout()
plt.savefig(os.path.join(figures_directory,'travis_county_non_numeric_metadata_histogram.pdf'),transparent=True, bbox_inches='tight')
plt.close()

In [3]:
# schedules
schedule_columns = database.query_table("""PRAGMA table_info(schedule)""")
schedule_columns = schedule_columns[~schedule_columns['name'].isin(['metadata_id','timestep'])]['name'].tolist()

for i, column in enumerate(schedule_columns):
    print(f'\r{i+1}/{len(schedule_columns)}',end='')
    database.query_table(f"""
    SELECT
        metadata_id,
        day*24 + hour AS timestep,
        day,
        hour,
        AVG({column}) AS {column}
    FROM (
        SELECT
            metadata_id,
            CAST((timestep - CAST(timestep/96 AS INTEGER)*96)/4 AS INTEGER) AS hour,
            CAST(timestep/96 AS INTEGER) AS day,
            {column}
        FROM schedule
    ) t
    GROUP BY
        metadata_id,
        day,
        hour
    """).to_pickle(os.path.join(schedule_data_directory,f'{column}.pkl'))

24/24